# Pipeline de entrenamiento y predicción

En este notebook se ejemplifica como ejecutar los scripts para poder:
    
1. Entrenar un modelo para clasificar las zonas de un mapa
2. Usar ese mismo modelo para analizar zonas y tener un estimativo de las zonas 

## Preparando los datos

En este notebook se demostrará el proceso utilizando un recorte de imágenes SPOT6/7 para la ciudad de Concordia, Entre Ríos. Se utilizará la imagen correspondiente al año 2017 para entrenar el modelo, y la del 2018 para evaluarlo.

In [3]:
import os

root_path = '/ap-siu-habitat'

images_path = os.path.join(root_path, 'data', 'imagenes')
train_tif_path = os.path.join(images_path, 'concordia_2016', 'concordia_small_2016.tif')
test_tif_path = os.path.join(images_path, 'concordia_2018', 'concordia_small_2018.tif')

*Opcional*: Si quiere ejecutar el proceso para las imagenes enteras, descomente las lineas de la siguiente celda y ejecútela.

**Tenga en cuenta que el entrenamiento puede demorar varias horas.**

In [2]:
def download_file(url, out_path):
    if os.path.exists(out_path):
        print("El archivo {} ya existe. Si quiere descargarlo de nuevo, debe borrarlo primero.".format(out_path))
        return

    #!wget -O $test_tif_path $test_remote_tif_url
    r = requests.get(url, stream=True) 
    with open(out_path, "wb") as f: 
        i = 0
        for chunk in r.iter_content(chunk_size=available_mem): 
            # writing one chunk at a time
            if chunk: 
                f.write(chunk)
                clear_output(wait=True)
                print("Escribiendo bloque {} de test, de {} bytes".format(i,available_mem))
                i += 1

In [3]:
# concordia_data_path = '{}/data/conae/centro/concordia'.format(root_path)

# train_product_folder = 'train'
# train_product_image = 'concordia_2016.TIF'
# train_tif_path = '{}/{}/{}'.format(concordia_data_path,train_product_folder,train_product_image)
# train_remote_tif_url = 'https://storage.googleapis.com/dym-ap-siu-habitat-public-images/concordia/IMG_SPOT7_PMS_201601281334024_ORT_C0000000058530_R1C1.TIF'

# !wget --continue -O $train_tif_path $train_remote_tif_url

# test_product_folder = 'test'
# test_product_image = 'concordia_2018.TIF'
# test_tif_path = '{}/{}/{}'.format(concordia_data_path,test_product_folder,test_product_image)
# test_remote_tif_url = 'https://storage.googleapis.com/dym-ap-siu-habitat-public-images/concordia/IMG_SPOT6_PMS_201801071323410_ORT_C0000000058510_R1C1.TIF'

# !wget --continue -O $test_tif_path $test_remote_tif_url

Inicializamos el resto de las variables para ejecutar los scripts, entre ellas la cantidad de hilos paralelos (`n_jobs`) , arboles (`trees`) y profundidad de los mismos (`depth`). Además indicamos las rutas de los scripts, archivos vectoriales etiquetados y rutas de salida.

In [3]:
script_train_path = '{}/script/siu_train.py'.format(root_path)
script_test_path = '{}/script/siu_test.py'.format(root_path)

n_jobs = 4
trees = 107
depth = 1007

train_shp_path = '{}/data/etiquetado/centro/concordia2016_train.shp'.format(root_path)
out_train_model_path = '{}/data/concordia2016_train_rf_t{}_d{}'.format(root_path,trees,depth)
model_file_name = 'concordia2016_train_rf_t{trs}_d{dpt}_rf_t{trs}_d{dpt}.txt'.format(trs=trees,dpt=depth)

test_shp_path = '{}/data/etiquetado/centro/concordia2018_train.shp'.format(root_path)
out_test_results_path = '{}/data/concordia2018_train_rf_t{}_d{}'.format(root_path,trees,depth)

out_train_model_path_full = '{}/model_train_rf_t{}_d{}/{}'.format(out_train_model_path,trees,depth,model_file_name)
out_test_results_path_train = '{}/data/concordia2016_test_rf_t{}_d{}'.format(root_path,trees,depth)

## Entrenamiento del modelo

Entrenamos a modo de ejemplo sobre los datos de la ciudad de concordia del año 2016

In [10]:
# Para ejecutarlo en el notebook, descomentar y usar la siguente linea 
#!python3 $script_train_path $train_tif_path $train_shp_path $out_train_model_path $n_jobs $trees $depth --root_path $root_path

# Para ejecutarlo en background independiente del notbook usar la siguente linea
!python3 $script_train_path $train_tif_path $train_shp_path $out_train_model_path $n_jobs $trees $depth --root_path $root_path >> ./concordia2016_train_rf.log 2>&1

In [11]:
print("Entrenamiento terminado")

Entrenamiento terminado


Podemos analizar la carpeta de salida donde encontramos el resultado de performance y el modelo obtenido

In [4]:
out_folder = '{}/model_train_rf_t{}_d{}/'.format(out_train_model_path,trees,depth)
# !ls -lth $out_folder
os.listdir(out_folder)

['concordia2016_train_rf_t107_d1007_rf_t107_d1007_acc.txt',
 'concordia2016_train_rf_t107_d1007_rf_t107_d1007.txt']

Abriendo el archivo terminado en `_acc.txt` vemos la matriz de confusión y las medidas de performance de entrenamiento

In [5]:
performance_file = '{}{}'.format(out_folder,'concordia2016_train_rf_t107_d1007_rf_t107_d1007_acc.txt')
#!cat $performance_file
with open(performance_file, 'r') as f:
    print(f.read())

Error Matrix

                Observed
                --------
                C     4   C     5   Total   User(%)
                -------   -------   -----   -------
Predicted C004| (49)      0         49      100.00
          C005| 0         (365)     365     100.00
         Total| 49        365       (414)
   Producer(%)| 100.00    100.00            (100.00%)

Samples: 414

Statistics

Overall Accuracy (%): 100.00
Kappa: 1.00
F-beta: 1.00
Hamming loss: 0.00

Class report

             precision    recall  f1-score   support

          4       1.00      1.00      1.00        49
          5       1.00      1.00      1.00       365

avg / total       1.00      1.00      1.00       414



## Predicción usando el modelo aprendido

Usamos el modelo aprendido para analizar la misma zona pero del año 2018


In [14]:
# Para ejecutarlo en el notebook, descomentar y usar la siguente linea 
#!python3 $script_test_path $test_tif_path $test_shp_path $out_train_model_path_full $out_test_results_path $n_jobs $trees $depth --root_path $root_path
#!python3 $script_test_path $train_tif_path $train_shp_path $out_train_model_path_full $out_test_results_path_train $n_jobs $trees $depth --root_path $root_path

# Para ejecutarlo en background independiente del notbook usar la siguente linea
# Sobre el test set (2018)
!python3 $script_test_path $test_tif_path $test_shp_path $out_train_model_path_full $out_test_results_path $n_jobs $trees $depth --root_path $root_path >> ./concordia2018_test_rf.log 2>&1
# Sobre el train set (2016)
!python3 $script_test_path $train_tif_path $train_shp_path $out_train_model_path_full $out_test_results_path_train $n_jobs $trees $depth --root_path $root_path >> ./concordia2016_test_train_rf.log 2>&1

In [15]:
print("Evaluacion terminada")

Evaluacion terminada


La performance sobre el año 2018 usado de test es la siguiente

In [6]:
out_folder = '{}/model_test_rf_t{}_d{}/'.format(out_test_results_path,trees,depth)
performance_file = '{}{}'.format(out_folder, 'concordia2018_train_rf_t107_d1007_rf_t107_d1007_acc.txt')
# !cat $performance_file
with open(performance_file, 'r') as f:
    print(f.read())

Error Matrix

                Observed
                --------
                C     2   C     4   C     5   Total   User(%)
                -------   -------   -------   -----   -------
Predicted C002| (0)       0         0         0       0.00
          C004| 121       (0)       2         123     0.00
          C005| 0         0         (822)     822     100.00
         Total| 121       0         824       (945)
   Producer(%)| 0.00      0.00      99.76             (86.98%)

Samples: 945

Statistics

Overall Accuracy (%): 86.98
Kappa: 0.46
Hamming loss: 0.13

Class report

             precision    recall  f1-score   support

          2       0.00      0.00      0.00       121
          4       0.00      0.00      0.00         0
          5       1.00      1.00      1.00       824

avg / total       0.87      0.87      0.87       945



Vemos que en la carpeta `clasificacion_...` dentro de la carpeta de salida, está la imagen obtenida luego de aplicar el modelo

In [7]:
map_file = '{}/clasificacion_test_rf_t{}_d{}/concordia2018_train_rf_t107_d1007_rf_t107_d1007.tif'.format(out_test_results_path, trees, depth)
# !ls $map_file
os.listdir(os.path.dirname(map_file))

['concordia2018_train_rf_t107_d1007_rf_t107_d1007.tif']

## Mover los resultados a una carpeta con formato esperado para post-procesamiento

Movemos los resultados a una estructura de carpetas con el patrón `region/ciudad/año` que esperan los scripts de post-procesamiento 

In [16]:
import subprocess
import glob
import shutil

def prepare_results(*, year, path_salida, region_city, resultados_classifier, imgs_orig, raster_virtual_path):
    # Crear el directorio
    results_full_path = os.path.join(path_salida, region_city, str(year))
    print('Creando directorio {}'.format(results_full_path))
    os.makedirs(os.path.dirname(results_full_path), exist_ok=True)

    # Copiar clasificación a la carpeta
    for tif_file in glob.glob(os.path.join(resultados_classifier, '*.tif')):
        shutil.copy(tif_file,results_full_path)

    # Generar raster virtual .vrt con las imágenes originales
    original_images_full_path = os.path.join(raster_virtual_path, region_city)
    os.makedirs(os.path.join(root_path, original_images_full_path), exist_ok=True)

    cmd = "gdalbuildvrt {dst} {src}".format(
        dst=os.path.join(original_images_full_path, "{}.vrt".format(year)),
        src=os.path.join(imgs_orig, "*.tif"))
    print(cmd)

    process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE)
    print(process.stdout)

In [17]:
prepare_results(year='2016/',
                region_city='centro/concordia',
                path_salida='data/resultados/finales',
                resultados_classifier='data/concordia2016_train_rf_t107_d1007/clasificacion_train_rf_t107_d1007',
                imgs_orig='data/imagenes/concordia_2016',
                raster_virtual_path='data/imagenes_orig')

Creando directorio data/resultados/finales/centro/concordia/2016/
gdalbuildvrt data/imagenes_orig/centro/concordia/2016/.vrt data/imagenes/concordia_2016/*.tif
b'0...10...20...30...40...50...60...70...80...90...100 - done.\n'


In [18]:
prepare_results(year='2018/',
                region_city='centro/concordia',
                path_salida='data/resultados/finales',
                resultados_classifier='data/concordia2018_train_rf_t107_d1007/clasificacion_test_rf_t107_d1007',
                imgs_orig='data/imagenes/concordia_2018',
                raster_virtual_path='data/imagenes_orig')

Creando directorio data/resultados/finales/centro/concordia/2018/
gdalbuildvrt data/imagenes_orig/centro/concordia/2018/.vrt data/imagenes/concordia_2018/*.tif
b'0...10...20...30...40...50...60...70...80...90...100 - done.\n'
